# MoveNet_FPN 训练笔记

## 导入工程

In [ ]:
# 导入系统库
import os
import timm
from tqdm import tqdm

# 导入sparrow
from sparrow.models.movenet_fpn import MoveNet_FPN
from sparrow.datasets.coco_kpts import create_kpts_dataloader
from sparrow.losses.movenet_loss import MoveNetLoss
from sparrow.utils.torch_utils import EMA, evaluate, visualize_predictions
from torch.optim.lr_scheduler import CosineAnnealingLR

# 导入torch库
import torch

## 参数设置

### 系统参数

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
INPUT_SIZE = 192
BATCH_SIZE = 8
NUM_WORKERS = 4
ANCHOR_SIZES = [32, 64, 128, 256, 512]
ANCHOR_RATIOS = [0.5, 1.0, 2.0, 1/3.0, 3.0]

WEIGHTS_DIR = "./outputs/ssdlite" # 保存权重的目录
TEST_IMAGE_PATH = "./res/india_road.png" # 你的测试图片路径

## 创建模型

In [ ]:
backbone_fpn = timm.create_model('mobilenetv3_large_100', pretrained=True, features_only=True, out_indices=(2, 3, 4))
model_fpn = SSDLite_FPN(backbone_fpn, num_classes=NUM_CLASSES, fpn_out_channels=128, num_anchors=len(ANCHOR_RATIOS))
model_fpn.to(device)

# EMA评估器
ema = EMA(model_fpn)